In [ ]:
try:
    import google.colab
    !rm -rf /content/SportWarzSim
    !git clone https://github.com/daltontf/SportWarzSim.git 
    %cd SportWarzSim
except ImportError:
    pass

In [ ]:
from sportwarz import LeaguesModel

leagues_model = LeaguesModel()

leagues_model.load_counties_geojson()

In [ ]:
leagues_model.load_counties_data()

### Blank Sandbox

In [ ]:
leagues_model.reset_county_styles()

leagues_model.render_map(None)

In [ ]:
from sportwarz import LeagueJson, LeaguesModel, league_teams_sums

import ast
import json
import codecs
import ipywidgets as widgets
import pandas as pd

import panel as pn

pn.extension('ace', 'jsoneditor')

pd.set_option('display.float_format', lambda x: '%.0f' % x)
pd.set_option('display.max_rows', 256)
pd.set_option('display.width', 256)

leagues = { }

error_message = widgets.Label(style = { "color" : "red" })

out = widgets.Output()
out.clear_output()

load_league_select = widgets.Dropdown(description = "League to Load:",
                                      options = ["", "NFL", "MLB", "NBA", "NHL", "MLS"],
                                      style={'description_width': '120px'})
# load_league_json_editor = pn.widgets.JSONEditor(width=400)
load_league_json_editor = widgets.Textarea(value = "", description="JSON:", style={'description_width': '60px'}, layout=widgets.Layout(width="400px", height="300px"))

load_league_button = widgets.Button(description="Load League")

eval_textarea = widgets.Textarea(value = "", description="Python Code:", style={'description_width': '60px'}, layout=widgets.Layout(width="100%", height="400px"))
eval_execute_button = widgets.Button(description="Execute Code")

# layout = pn.Column(
#     pn.panel(tab),
#     load_league_json_editor,
#     pn.panel(error_message)
# )
# layout

load_league_panel = widgets.GridBox(children = [
    load_league_select, widgets.Label("Load JSON data stored on server"),
    load_league_json_editor, widgets.Label("Edit JSON before load?"),
    load_league_button
], layout= widgets.Layout(
        grid_template_columns="repeat(2, 400px)",
        grid_gap="10px 10px")
)

eval_panel = widgets.VBox([
    eval_textarea,
    eval_execute_button
])

tab = widgets.Tab()
tab.children = [load_league_panel, eval_panel]
tab.titles =   ["Load League", "Evaluate Python"]

league_dropdown_options = []

@out.capture()
def lock_tabs(locked=True, widget=tab):
    if hasattr(widget, "children"):
        for child in widget.children:
            lock_tabs(locked, child)
    elif hasattr(widget, "disabled"):
        widget.disabled = locked

@out.capture()
def show_pre_post_merged_results(league_name:str, before_model:LeaguesModel, after_model: LeaguesModel):
    pre_sums = league_teams_sums(before_model._leagues[league_name])
    post_sums = league_teams_sums(after_model._leagues[league_name])
    merged = pd.merge(pre_sums, post_sums, on='team_name', how='outer', suffixes=("_before", "_after"))
    merged["share_population_before"] = merged["share_population_before"].fillna(0) 
    merged["share_population_after"] = merged["share_population_after"].fillna(0) 
    merged["share_population_value_before"] = merged["share_population_value_before"].fillna(0) 
    merged["share_population_value_after"] = merged["share_population_value_after"].fillna(0) 
    merged["share_population_delta"] = merged["share_population_after"] - merged["share_population_before"]
    merged["share_population_value_delta"] = merged["share_population_value_after"] - merged["share_population_value_before"]
    print(merged)
    print(f'Population Values Sums\t: {merged["share_population_value_before"].sum():,.0f}\t{merged["share_population_value_after"].sum():,.0f}')
    print(f'Population Values Sum Delta = {(merged["share_population_value_after"].sum() - merged["share_population_value_before"].sum()):,.0f}')        


@out.capture()
def load_league_select_change(change):
    out.clear_output() 
    league_name = load_league_select.value  
    if league_name:
        load_league_json_editor.value = leagues_model.read_league(league_name)

@out.capture()
def load_league(button):
    out.clear_output()
    lock_tabs(True)
    error_message.value = ""
    try:
        league_dict = json.loads(load_league_json_editor.value)
        league_name = league_dict["league_name"]  

        before_model = None      

        if league_name in leagues_model._leagues:
            before_model = leagues_model.copy_with_just_league(league_name)

        leagues_model.load_league(league_dict)     
        leagues_model.calculate_league_distances(league_name) 
        leagues_model.compute_league_shares(league_name)  
    
        leagues_model.heatmap_counties(league_name)
        leagues_model.refresh_geojson_layer()

        if before_model:
            show_pre_post_merged_results(league_name, before_model, leagues_model) 
      
        if league_name not in league_dropdown_options:
            league_dropdown_options.append(league_name)  
    except Exception as ex:
        print(ex)
    finally:
        lock_tabs(False)

def eval_execute(button):
    out.clear_output()
    lock_tabs(True)
    error_message.value = ""
    try:
        ast.parse(eval_textarea.value, mode="exec")
        exec(eval_textarea.value, {}, {
            "leagues_model": leagues_model,
            "show_pre_post_merged_results": show_pre_post_merged_results
            })
    except SyntaxError as e:
        print("Syntax error at line", e.lineno, "col", e.offset)
        print(e.text)
    except Exception as e:
        print("Eval runtime error:", e)
    finally:
        for league_name in load_league_select.options:
            if league_name in leagues_model._leagues and league_name not in league_dropdown_options:
                league_dropdown_options.append(league_name)  
        lock_tabs(False)  

load_league_button.on_click(load_league)
load_league_select.observe(load_league_select_change, names='value')


eval_execute_button.on_click(eval_execute)

widgets.VBox(children = [
  tab,
  error_message  
])

In [ ]:
out